In [1]:
using DataFrames, Logging, Revise
global_logger(ConsoleLogger(stdout, Logging.Info));

# MOFun Find-Replace

### Import the module and prepare the input data

In [2]:
using PorousMaterials, MOFun
s_moty = moiety("find-replace/2-!-p-phenylene")
r_moty = moiety("2-acetylamido-p-phenylene")
parent = Crystal("IRMOF-1.cif")
strip_numbers_from_atom_labels!(parent)
infer_bonds!(parent, true)

### Run the search

In [3]:
search = substructure_search(s_moty, parent);

### See the results

The `Search` object has attributes `query` and `results`

In [4]:
search.query;

In [5]:
search.results;

Several functions are provided to query the results

In [6]:
nb_isomorphisms(search);

In [7]:
nb_locations(search);

In [8]:
nb_configs_at_loc(search);

### Make a set number of random replacements

In [9]:
# returns ids of results corresponding to the 
function get_config_ids(search::Search; orientations::Array{Int}=Int[], nb_loc::Int=0, locations::Array{Int}=Int[])::Array{Int}
    # randomize orientations
    orientations = orientations == [] ? [rand(1:n) for n ∈ nb_configs_at_loc(search)] : orientations
    nb_loc = nb_loc == 0 ? nb_locations(search) : nb_loc
    locations = locations == [] ? [1:nb_locations(search)...] : locations
    # get id of ith isom in each group per orientations array
    return [loc.id[orientations[i]] for (i, loc) ∈ enumerate(groupby(search.results, :p_subset)) if i ∈ locations]
end

get_config_ids(search);

In [10]:
substructure_replace(s_moty, r_moty, parent, search, get_config_ids(search, nb_loc=1));

### Make a random replacement at each location

In [11]:
substructure_replace(s_moty, r_moty, parent, search, get_config_ids(search));

### Make a random replacement at selected locations

In [12]:
locations = [2, 3, 5, 7]
substructure_replace(s_moty, r_moty, parent, search, get_config_ids(search, locations=locations));

### Make a specific series of replacements

In [13]:
orientations = [1 for _ ∈ 1:nb_locations(search)]
substructure_replace(s_moty, r_moty, parent, search, get_config_ids(search, orientations=orientations, locations=locations));